In [2]:
import pandas as pd

# Lectura de datos

In [16]:
df_caracteristicas_equipos = pd.read_csv('../data/Caracteristicas_Equipos.csv')
df_historicos_ordenes = pd.read_csv('../data/Historicos_Ordenes.csv')
df_registros_condiciones = pd.read_csv('../data/Registros_Condiciones.csv')

# 1. Preparación DF

In [17]:
def convertir_fecha_a_datetime(df, columna):
    df[columna] = pd.to_datetime(df[columna])
    return df

def cantidad_nulos_por_cada_columna(df, nombre_df):
    print ("--------------------------------------------------")
    print (f'Cantidad de nulos por cada columna de {nombre_df}')
    print (df.isnull().sum().to_frame('Total_Nulos').T)

def mostrar_duplicados_por_columna(df, nombre_columna, nombre_df):
    print ("--------------------------------------------------")
    duplicados = df[df.duplicated(subset=[nombre_columna])]
    print (f'Duplicados en la columna {nombre_columna} de {nombre_df}')
    print (duplicados)
    print (f'Cantidad de duplicados: {len(duplicados)}')
    # Eliminar duplicados y mantener la primera ocurrencia
    df_sin_duplicados = df.drop_duplicates(subset=[nombre_columna], keep='first')
    print(f'\nSe eliminaron {len(df) - len(df_sin_duplicados)} registros duplicados')
    return df_sin_duplicados

### Se convierten a fechas los campos Fecha de cada DF

In [18]:
df_historicos_ordenes = convertir_fecha_a_datetime(df_historicos_ordenes, 'Fecha')
df_registros_condiciones = convertir_fecha_a_datetime(df_registros_condiciones, 'Fecha')

### Se comprueban los nulos
### Se observa que los nulos afectarán al modelo. !No se eliminan!!

In [ ]:
cantidad_nulos_por_cada_columna(df_caracteristicas_equipos, 'df_caracteristicas_equipos')
print ()
cantidad_nulos_por_cada_columna(df_historicos_ordenes, 'df_historicos_ordenes')
print()
cantidad_nulos_por_cada_columna(df_registros_condiciones, 'df_registros_condiciones')

### Se muestran los duplicados y se eliminan

In [ ]:
mostrar_duplicados_por_columna(df_caracteristicas_equipos, 'ID_Equipo', 'df_caracteristicas_equipos')
mostrar_duplicados_por_columna(df_historicos_ordenes, 'ID_Orden', 'df_historicos_ordenes')
mostrar_duplicados_por_columna(df_registros_condiciones, 'ID_Registro', 'df_registros_condiciones')

### 1. Explorar usando YDATA PROFILING


In [ ]:
from ydata_profiling import ProfileReport


profile = ProfileReport(df_caracteristicas_equipos, title="Caracteristicas Equipos Profiling Report",
                        correlations={
                            "pearson": {"calculate": True},
                            "cramers": {"calculate": True},
                        })
profile.to_notebook_iframe()

## Creación features

### Añadimos a cada orden el número de horas recomendas para el ID_Equipo

In [21]:
# Hacemos el merge usando 'ID_Equipo' y eliminamos duplicados
df_trabajo_modelo = df_historicos_ordenes.merge(
    df_caracteristicas_equipos[['ID_Equipo', 'Horas_Recomendadas_Revision']],
    on='ID_Equipo',
    how='left'  # Utilizamos left join para mantener todos los registros del primer DataFrame
).drop_duplicates(subset=['ID_Equipo', 'Fecha'])

### Añadirmos el tiempo transcurrido en horas desde el último mantenimiento

In [22]:
# Ordenamos el DataFrame por ID_Equipo y Fecha
df_trabajo_modelo = df_trabajo_modelo.sort_values(['ID_Equipo', 'Fecha'])

# Calculamos la diferencia de tiempo con la orden anterior para cada equipo
df_trabajo_modelo['Horas_Desde_Ultima_Orden'] = df_trabajo_modelo.groupby('ID_Equipo')['Fecha'].diff().dt.total_seconds() / 3600

# Reemplazamos los NaN (que corresponden a la primera orden de cada equipo) con 0
df_trabajo_modelo['Horas_Desde_Ultima_Orden'] = df_trabajo_modelo['Horas_Desde_Ultima_Orden'].fillna(0)

### Añadimos las condiciones en el momento del mantenimiento usando las medidas más próximas

In [ ]:

# Preparamos el DataFrame de registros de condiciones seleccionando solo las columnas necesarias
df_condiciones_reducido = df_registros_condiciones[['Fecha', 'ID_Equipo', 'Temperatura_C', 'Vibracion_mm_s', 'Horas_Operativas']]

# Realizamos un merge asíncrono para encontrar los registros más cercanos en tiempo
df_trabajo_modelo = pd.merge_asof(
    df_trabajo_modelo.sort_values('Fecha'),
    df_condiciones_reducido.sort_values('Fecha'),
    by='ID_Equipo',
    on='Fecha',
    direction='nearest'
)
print (df_trabajo_modelo.head())


## Candidatos

1. `LogisticRegression` # No coincide con perfil de datos, para comparar
2. `RandomForestClassifier` 
3. `GradientBoostingClassifier`

In [68]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer


In [106]:
target_column = "Tipo_Mantenimiento"

# Separar características y target
X = df_trabajo_modelo.drop(target_column, axis=1)
X = X.drop(columns=['Fecha'])
#y = pd.get_dummies(df_trabajo_modelo[target_column], drop_first=True)
y = pd.get_dummies(df_trabajo_modelo[target_column], drop_first=True).values.reshape(-1, 1).ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [105]:
print (y)

['Preventivo' 'Preventivo' 'Preventivo' ... 'Preventivo' 'Preventivo'
 'Preventivo']


### Pipelines

In [98]:
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(X)
categorical_columns = categorical_columns_selector(X)

print("Numerical columns", numerical_columns)
print("Categorical columns", categorical_columns)

Numerical columns ['ID_Orden', 'ID_Equipo', 'Costo_Mantenimiento', 'Duracion_Horas', 'Horas_Recomendadas_Revision', 'Horas_Desde_Ultima_Orden', 'Temperatura_C', 'Vibracion_mm_s', 'Horas_Operativas']
Categorical columns ['Ubicacion']


In [99]:

preprocessor = ColumnTransformer(
    [
        ('numerical', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numerical_columns),
        ('categorical', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_columns)
    ])

pipelines = {
    "LogisticRegression": Pipeline([('preprocessor', preprocessor),('classifier', LogisticRegression())]),
    "RandomForestClassifier": Pipeline([('preprocessor', preprocessor),('classifier', RandomForestClassifier())]),
    "GradientBoostingClassifier": Pipeline([('preprocessor', preprocessor),('classifier', GradientBoostingClassifier())]),
}

In [102]:
def cv_train(name, pipeline, cv):
    print(f"Entrenar {name}")
    cv_results = cross_validate(pipeline, X_train, y_train, cv=cv, scoring="accuracy", return_estimator=True, return_train_score=True)
    trained_model = cv_results["estimator"][0]
    scores = pd.DataFrame(cv_results)

    print("test score (mean-std): {0:.2f} - {1:.2f}".format(scores["test_score"].mean(), scores["test_score"].std()))
    print("train score (mean-std): {0:.2f} - {1:.2f}".format(scores["train_score"].mean(), scores["train_score"].std()))
    print("params:", pipeline.named_steps.get("classifier").get_params())

    y_pred = trained_model.predict(X_test)
    f1 = f1_score(y_test, y_pred)

    return {"acc": round(scores["test_score"].mean(), 2), "f1": round(f1, 2),}

In [103]:
cvss = ShuffleSplit(n_splits=40, test_size=0.2, random_state=0)

results = {}

for name, pipeline in pipelines.items():
    results[name] = cv_train(name, pipeline, cvss)

print("Resultados:")
results_df=pd.DataFrame(results)
results_df

Entrenar LogisticRegression


c:\Users\daekka\anaconda3\envs\NTTData\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\daekka\anaconda3\envs\NTTData\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\daekka\anaconda3\envs\NTTData\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\daekka\anaconda3\envs\NTTData\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array w

test score (mean-std): 0.50 - 0.01
train score (mean-std): 0.51 - 0.00
params: {'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
Entrenar RandomForestClassifier


c:\Users\daekka\anaconda3\envs\NTTData\lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


KeyboardInterrupt: 